In [1]:
import numpy as np
import pandas as pd

from dpipe.io import load
from dl2021.utils import choose_root

In [2]:
from dl2021.paths import WMH_UNET3D_BASELINE_PATH as path_base_unet
from dl2021.paths import WMH_DM_BASELINE_PATH as path_base_dm


path_oracle_dm = choose_root(
    '/gpfs/data/gpfs0/b.shirokikh/experiments/da/spottune/wmh/baseline/wmh_dm_oracle/',
    '/nmnt/x4-hdd/experiments/da_exps/wmh/baseline/wmh_dm_oracle/',
)

path_oracle_unet = choose_root(
    '/gpfs/data/gpfs0/b.shirokikh/experiments/da/spottune/wmh/baseline/wmh_unet3d_oracle/',
    '/nmnt/x4-hdd/experiments/da_exps/wmh/baseline/wmh_dm_oracle/',
)

meta_path = choose_root(
    '/nmnt/x3-hdd/data/da_mri/wmh_ants/meta.csv',
    '/gpfs/data/gpfs0/b.shirokikh/data/wmh_ants/meta.csv',
)

meta = pd.read_csv(meta_path, index_col='id')
meta.head()

,flair,brain_mask,wmh,t1,fold,cite,x,y,z
id,,,,,,,,,
sub-Amst106,sub-Amst106/FLAIR.nii.gz,sub-Amst106/bm.nii.gz,sub-Amst106/wmh.nii.gz,sub-Amst106/T1.nii.gz,0,Amst,1.200000,0.976600,3.000000
sub-Utr23,sub-Utr23/FLAIR.nii.gz,sub-Utr23/bm.nii.gz,sub-Utr23/wmh.nii.gz,sub-Utr23/T1.nii.gz,2,Utr,0.958333,0.958333,2.999997
sub-Utr25,sub-Utr25/FLAIR.nii.gz,sub-Utr25/bm.nii.gz,sub-Utr25/wmh.nii.gz,sub-Utr25/T1.nii.gz,2,Utr,0.958333,0.958333,3.000000
sub-Sing56,sub-Sing56/FLAIR.nii.gz,sub-Sing56/bm.nii.gz,sub-Sing56/wmh.nii.gz,sub-Sing56/T1.nii.gz,1,Sing,1.000000,1.000000,2.999999
sub-Utr0,sub-Utr0/FLAIR.nii.gz,sub-Utr0/bm.nii.gz,sub-Utr0/wmh.nii.gz,sub-Utr0/T1.nii.gz,2,Utr,0.958333,0.958333,2.999999


In [3]:
def paths2results(path_base, path_oracle, meta=meta, n_val_oracle=5):
    records = []
    for s in sorted(meta['fold'].unique()):
        res_row = {}

        # one2all results:
        dices = load(path_base / f'experiment_{s}/test_metrics/dice_score.json')
        for t in sorted(set(meta['fold'].unique()) - {s}):
            df_row = meta[meta['fold'] == t].iloc[0]
            target_name = df_row['cite']

            ids_t = meta[meta['fold'] == t].index
            res_arr = [dsc for _id, dsc in dices.items() if _id in ids_t]
            
            res_row[target_name] = f'{np.mean(res_arr):.2f} ± {np.std(res_arr):.2f}'

        df_row = meta[meta['fold'] == s].iloc[0]
        source_name = df_row['cite']  
        dices = {}
        for n_val in range(n_val_oracle):
            dices = {**dices,
                     **load(path_oracle / f'experiment_{s * n_val_oracle + n_val}/test_metrics/dice_score.json')}
        res_arr = list(dices.values())
        res_row[source_name] = f'{np.mean(res_arr):.2f} ± {np.std(res_arr):.2f}'

        res_row[' '] = source_name
        records.append(res_row)
        
    df = pd.DataFrame.from_records(records, index=' ')
    return df

### U-Net 3D

In [4]:
df = paths2results(path_base_unet, path_oracle_unet)
df[df.index]

,Amst,Sing,Utr
,,,
Amst,0.57 ± 0.11,0.65 ± 0.17,0.00 ± 0.00
Sing,0.50 ± 0.16,0.68 ± 0.18,0.00 ± 0.00
Utr,0.00 ± 0.00,0.00 ± 0.00,0.69 ± 0.20


### DeepMedic

In [5]:
df = paths2results(path_base_dm, path_oracle_dm)
df[df.index]

,Amst,Sing,Utr
,,,
Amst,0.41 ± 0.12,0.51 ± 0.18,0.00 ± 0.00
Sing,0.31 ± 0.20,0.54 ± 0.21,0.00 ± 0.00
Utr,0.07 ± 0.05,0.09 ± 0.05,0.55 ± 0.24
